In [49]:
#@title Cell 1: Cài đặt toàn bộ (Thư viện, Code đã cung cấp & Hàm tiện ích)
# --- 1. Cài đặt và Imports ---
!pip install timm -q
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import timm
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
import os
import time

print(f"PyTorch version: {torch.__version__}")
print(f"Timm version: {timm.__version__}")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

# Tạo thư mục để lưu checkpoints
os.makedirs('checkpoints', exist_ok=True)

# --- 2. Các hàm tiện ích còn thiếu ---
def calculate_accuracy(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return 100. * correct / total

def save_checkpoint(model, optimizer, epoch, acc, path, is_best):
    state = {'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch, 'acc': acc}
    torch.save(state, path)
    if is_best:
        best_path = os.path.join(os.path.dirname(path), f'best_{os.path.basename(path)}')
        torch.save(state, best_path)
        print(f"🌟 New best model saved to {best_path} with accuracy {acc:.2f}%")

def evaluate_comprehensive(model, dataloader, device, model_name="Model"):
    print(f"\n🔬 Comprehensive Evaluation for {model_name}...")
    acc = calculate_accuracy(model, dataloader, device)
    # Thêm các metric khác nếu cần (ví dụ: F1, Precision, Recall)
    results = {'accuracy': acc}
    print(f"  - Accuracy: {acc:.2f}%")
    return results

def plot_history(history, title):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['test_acc'], label='Test Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# --- 3. Dán toàn bộ code bạn đã cung cấp vào đây ---
# (create_mobilevit_model, setup_finetuning, test_model_shapes,
#  get_cifar10_loaders, train_model, run_full_training)
# Cell 2: Model Creation and Finetuning Setup
# ========================================
def create_mobilevit_model(variant='mobilevit_xxs', num_classes=10, pretrained=True):
    """Create MobileViT model using timm library with ImageNet pretrained weights"""
    try:
        model = timm.create_model(variant, pretrained=pretrained, num_classes=num_classes)
        print(f"✅ Created {variant} (pretrained={pretrained}) with {sum(p.numel() for p in model.parameters()):,} parameters")
        return model, variant
    except:
        print(f"⚠️ {variant} not available, trying alternatives...")
        alternatives = ['mobilenetv3_small_100', 'efficientnet_b0', 'resnet18']
        for alt in alternatives:
            try:
                model = timm.create_model(alt, pretrained=pretrained, num_classes=num_classes)
                print(f"✅ Using {alt} as fallback (pretrained={pretrained})")
                return model, alt
            except:
                continue
        model = timm.create_model('resnet18', pretrained=False, num_classes=num_classes)
        print(f"✅ Using ResNet18 without pretrain as last resort")
        return model, 'resnet18'

def setup_finetuning(model, model_name, freeze_backbone=False):
    """Setup model for finetuning"""
    if freeze_backbone:
        print("🔒 Freezing backbone for feature extractor training...")
        for name, param in model.named_parameters():
            if 'classifier' not in name and 'head' not in name:
                param.requires_grad = False
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"📊 Model setup:")
    print(f"   Total params: {total_params:,}")
    print(f"   Trainable params: {trainable_params:,} ({100*trainable_params/total_params:.1f}%)")
    return model

# Cell 3: Data Loading
# ========================================
def get_cifar10_loaders(batch_size=128, image_size=32):
    """Get CIFAR-10 data loaders with strong augmentation"""
    padding = 4 if image_size == 32 else 0
    transform_train = transforms.Compose([
        transforms.Resize(image_size),
        transforms.RandomCrop(image_size, padding=padding),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.3, 0.3, 0.3, 0.2),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        transforms.RandomErasing(p=0.25)
    ])
    transform_test = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
    trainset = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    return trainloader, testloader

# Cell 5: Enhanced Training Function for Finetuning
def train_model(model, model_name, trainloader, testloader, device, epochs=100, lr=0.001, save_freq=20, freeze_epochs=0):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=lr*0.01)
    history = {'train_acc': [], 'test_acc': [], 'train_loss': [], 'lr': []}
    best_acc = 0
    
    print(f"🚀 Finetuning {model_name} for {epochs} epochs on CIFAR-10...")

    for epoch in range(epochs):
        if epoch < freeze_epochs:
            model = setup_finetuning(model, model_name, freeze_backbone=True)
        elif epoch == freeze_epochs:
            print(f"🔓 Unfreezing backbone at epoch {epoch+1}")
            model = setup_finetuning(model, model_name, freeze_backbone=False)
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr*0.1, weight_decay=0.01)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs-freeze_epochs, eta_min=lr*0.001)

        model.train()
        train_loss = correct = total = 0
        pbar = tqdm(trainloader, desc=f'Epoch {epoch+1}/{epochs} [Train]', leave=False)
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            current_lr = optimizer.param_groups[0]['lr']
            pbar.set_postfix({'Acc': f'{100.*correct/total:.1f}%', 'LR': f'{current_lr:.1e}'})
        
        train_acc = 100. * correct / total
        avg_train_loss = train_loss / len(trainloader)
        test_acc = calculate_accuracy(model, testloader, device)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        history['train_loss'].append(avg_train_loss)
        history['lr'].append(current_lr)
        
        is_best = test_acc > best_acc
        if is_best:
            best_acc = test_acc
        
        checkpoint_path = f'checkpoints/{model_name}_epoch_{epoch+1}.pth'
        save_checkpoint(model, optimizer, epoch + 1, test_acc, checkpoint_path, is_best)
        scheduler.step()
        status = "🌟" if is_best else ""
        print(f"Epoch {epoch+1:3d}/{epochs}: Train {train_acc:5.1f}% | Test {test_acc:5.1f}% {status}")
        
    print(f"✅ Finetuning complete! Best accuracy: {best_acc:.2f}%")
    return model, history, best_acc

# Cell 8: Full Finetuning Pipeline
def run_full_training(model_variant, train_loader, test_loader, epochs=10, freeze_epochs=0, pretrained=True, run_name=""):
    print(f"\n{'='*20} STARTING RUN: {run_name} {'='*20}")
    model, actual_variant = create_mobilevit_model(model_variant, pretrained=pretrained)
    
    model, history, best_acc = train_model(
        model, actual_variant, train_loader, test_loader, DEVICE, 
        epochs=epochs, lr=0.001, freeze_epochs=freeze_epochs
    )
    print(f"{'='*20} FINISHED RUN: {run_name} | Best Acc: {best_acc:.2f}% {'='*20}")
    return {'run_name': run_name, 'model': actual_variant, 'best_acc': best_acc, 'history': history}

print("\n✅ Tất cả các hàm đã được định nghĩa.")


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
PyTorch version: 2.8.0+cu126
Timm version: 1.0.19
Using device: cuda

✅ Tất cả các hàm đã được định nghĩa.


## Thử nghiệm 1: Giá trị của Trọng số Tiền huấn luyện (Pretrained Weights)

In [50]:
#@title Cell 2: Chạy Thử nghiệm 1
EPOCHS = 15 # Dùng số epoch nhỏ để chạy nhanh, kết quả thực tế cần nhiều hơn
results = []
train_loader_32, test_loader_32 = get_cifar10_loaders(image_size=32)

# --- Chạy với trọng số tiền huấn luyện ---
res_pretrained = run_full_training(
    model_variant='mobilevit_xs',
    train_loader=train_loader_32,
    test_loader=test_loader_32,
    epochs=EPOCHS,
    pretrained=True,
    run_name="MobileViT_XS_Pretrained"
)
results.append(res_pretrained)

# --- Chạy huấn luyện từ đầu ---
res_scratch = run_full_training(
    model_variant='mobilevit_xs',
    train_loader=train_loader_32,
    test_loader=test_loader_32,
    epochs=EPOCHS,
    pretrained=False,
    run_name="MobileViT_XS_From_Scratch"
)
results.append(res_scratch)


==================== STARTING RUN: MobileViT_XS_Pretrained ====================


model.safetensors:   0%|          | 0.00/9.34M [00:00<?, ?B/s]

✅ Created mobilevit_xs (pretrained=True) with 1,936,698 parameters
🚀 Finetuning mobilevit_xs for 15 epochs on CIFAR-10...
🔓 Unfreezing backbone at epoch 1
📊 Model setup:
   Total params: 1,936,698
   Trainable params: 1,936,698 (100.0%)


Epoch 1/15 [Train]:   0%|          | 0/391 [00:00<?, ?it/s]

🌟 New best model saved to checkpoints/best_mobilevit_xs_epoch_1.pth with accuracy 66.35%
Epoch   1/15: Train  45.5% | Test  66.3% 🌟


...

🌟 New best model saved to checkpoints/best_mobilevit_xs_epoch_13.pth with accuracy 51.72%
Epoch  13/15: Train  44.8% | Test  51.7% 🌟


Epoch 14/15 [Train]:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch  14/15: Train  44.9% | Test  51.6% 


Epoch 15/15 [Train]:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch  15/15: Train  45.3% | Test  51.5% 
✅ Finetuning complete! Best accuracy: 51.72%
==================== FINISHED RUN: MobileViT_XS_From_Scratch | Best Acc: 51.72% ====================


## Thử nghiệm 3: Ảnh hưởng của Độ phân giải Đầu vào

In [ ]:
#@title Cell 4: Chạy Thử nghiệm 3
# --- Tải dữ liệu với độ phân giải 64x64 ---
print("==> Preparing upscaled data (64x64)...")
train_loader_64, test_loader_64 = get_cifar10_loaders(image_size=64)
print("✅ Upscaled data ready.")

# --- Chạy với độ phân giải 32x32 (Baseline) ---
# Có thể dùng lại kết quả `res_e2e` ở trên nếu đã chạy
res_32 = run_full_training(
    model_variant='mobilevit_xs',
    train_loader=train_loader_32,
    test_loader=test_loader_32,
    epochs=EPOCHS,
    freeze_epochs=0,
    pretrained=True,
    run_name="Input_Resolution_32x32"
)
results.append(res_32)

# --- Chạy với độ phân giải 64x64 ---
res_64 = run_full_training(
    model_variant='mobilevit_xs',
    train_loader=train_loader_64,
    test_loader=test_loader_64,
    epochs=EPOCHS,
    freeze_epochs=0,
    pretrained=True,
    run_name="Input_Resolution_64x64"
)
results.append(res_64)

==> Preparing upscaled data (64x64)...
✅ Upscaled data ready.

==================== STARTING RUN: Input_Resolution_32x32 ====================
✅ Created mobilevit_xs (pretrained=True) with 1,936,698 parameters
🚀 Finetuning mobilevit_xs for 15 epochs on CIFAR-10...
🔓 Unfreezing backbone at epoch 1
📊 Model setup:
   Total params: 1,936,698
   Trainable params: 1,936,698 (100.0%)


Epoch 1/15 [Train]:   0%|          | 0/391 [00:00<?, ?it/s]

🌟 New best model saved to checkpoints/best_mobilevit_xs_epoch_1.pth with accuracy 68.25%
Epoch   1/15: Train  46.3% | Test  68.2% 🌟


Epoch 2/15 [Train]:   0%|          | 0/391 [00:00<?, ?it/s]

🌟 New best model saved to checkpoints/best_mobilevit_xs_epoch_2.pth with accuracy 77.32%
Epoch   2/15: Train  67.9% | Test  77.3% 🌟


...

Epoch 8/15 [Train]:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch   8/15: Train  91.0% | Test  91.7% 


Epoch 9/15 [Train]:   0%|          | 0/391 [00:00<?, ?it/s]

🌟 New best model saved to checkpoints/best_mobilevit_xs_epoch_9.pth with accuracy 91.90%
Epoch   9/15: Train  91.6% | Test  91.9% 🌟


Epoch 10/15 [Train]:   0%|          | 0/391 [00:00<?, ?it/s]